## Phase processing

Note: This notebook does not work on Windows, so if you are interested in learning about the phase processing techniques in Py-ART, try and find a buddy with a Linux or OS X Laptop!

This notebook will go over how to do basic phase processing with Py-ART. We will start with loading the required modules and load a radar data file.

In [ ]:
%pylab inline
import warnings
warnings.filterwarnings('ignore')
import pyart
import cartopy.crs as ccrs

In [ ]:
radar = pyart.io.read('../inf_tutorial_data/cfrad.20060120_005000.000_to_20060120_005810.000_CPOL_PPI_level1a.nc')
radar.fields.keys()

Let's view the radar file over a region of scattered convection. Here, we can see signatures in the differntial phase field that correspond to the convective regions, but overall the field looks quite noisy and needs some processing in order to make a field that is easier to interpret. Thankfully, Py-ART gives us the capability to do this.

In [ ]:
display = pyart.graph.RadarMapDisplayCartopy(radar)
fig = plt.figure(figsize=(15,5))
ax1 = plt.subplot(1,2,1, projection=ccrs.PlateCarree())
display.plot_ppi_map('DBZ', cmap='pyart_LangRainbow12', vmin=0, vmax=60,
                     resolution='10m', min_lat=-12., max_lat=-11., min_lon=130., max_lon=132.)
ax2 = plt.subplot(1,2,2, projection=ccrs.PlateCarree())
display.plot_ppi_map('PHIDP', cmap='pyart_LangRainbow12', vmin=-180, vmax=180,
                     resolution='10m', min_lat=-12., max_lat=-11., min_lon=130., max_lon=132.)

The phase processing technique in Py-ART requires a field called normalized coherent power. If you don't have this, it is standard practice to make a fake normalized coherent power field that is 1 where points are to be included in the processing and 0 where they are not. So, following the example in the dealiasing notebook, we will make a gatefilter based off the velocity texture to remove noise first and create the mask in the normalized coherent power field.

In [ ]:
texture_field = pyart.retrieve.calculate_velocity_texture(radar, nyq=13.8, vel_field='VEL')

radar.add_field('texture', texture_field, replace_existing=True)
fake_ncp_field = {}
fake_ncp_field['data'] = np.ma.where(texture_field['data'] > 3, 0, 1)
fake_ncp_field['long_name'] = 'Normalized Coherent Power'
fake_ncp_field['standard_name'] = 'normalized_coherent_power'
fake_ncp_field['units'] = ''
fake_ncp_field['coordinates'] = texture_field['coordinates']
radar.add_field('normalized_coherent_power', fake_ncp_field, replace_existing=True)

We will then run the phase processing and store the results in two new fields.

In [ ]:
reproc_phase, kdp = pyart.correct.phase_proc_lp(radar, 0, refl_field='DBZ', 
                                                rhv_field='RHOHV', phidp_field='PHIDP')
radar.add_field('corrected_differential_phase', reproc_phase, replace_existing=True)
radar.add_field('specific_differential_phase', kdp, replace_existing=True)

We now have a much smoother field of differential phase as well as specific differential phase, which tells us the gradient of differential phase along the ray. In particular, higher values of specific differential phase typically demonstrate regions with higher amounts of more oblate raindrops. The elevated regions of specific differential phase are commonly called "KDP columns."

In [ ]:
fig = plt.figure(figsize=(15,5))
ax1 = plt.subplot(1,2,1, projection=ccrs.PlateCarree())
display.plot_ppi_map('corrected_differential_phase', cmap='pyart_LangRainbow12', vmin=0, vmax=90,
                     resolution='10m', min_lat=-12., max_lat=-11., min_lon=130., max_lon=132.)
ax2 = plt.subplot(1,2,2, projection=ccrs.PlateCarree())
display.plot_ppi_map('specific_differential_phase', cmap='pyart_LangRainbow12', vmin=0, vmax=2,
                     resolution='10m', min_lat=-12., max_lat=-11., min_lon=130., max_lon=132.)

## Exercise

Repeat the above exercise, but use ../inf_tutorial_data/sgpxsaprcmacsurI5.c1.20170801.044013.nc. Notice here that there is a normalized coherent power field already in the radar. What can you say about the presence of oblate drops in this scan compared to the previous example?

In [ ]:
%load section2_answer.py